# test 1

In [1]:
import uproot
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle('input_df/input_rho_rho.pkl')

In [3]:
df.columns

Index(['pi_E_1_br', 'pi_px_1_br', 'pi_py_1_br', 'pi_pz_1_br', 'pi_E_2_br',
       'pi_px_2_br', 'pi_py_2_br', 'pi_pz_2_br', 'pi0_E_1_br', 'pi0_px_1_br',
       'pi0_py_1_br', 'pi0_pz_1_br', 'pi0_E_2_br', 'pi0_px_2_br',
       'pi0_py_2_br', 'pi0_pz_2_br', 'rho_E_1_br', 'rho_px_1_br',
       'rho_py_1_br', 'rho_pz_1_br', 'rho_E_2_br', 'rho_px_2_br',
       'rho_py_2_br', 'rho_pz_2_br', 'aco_angle_1', 'y_1_1', 'y_1_2', 'w_a',
       'w_b', 'm_1', 'm_2'],
      dtype='object')

In [8]:
df.head()

,pi_E_1_br,pi_px_1_br,pi_py_1_br,pi_pz_1_br,pi_E_2_br,pi_px_2_br,pi_py_2_br,pi_pz_2_br,pi0_E_1_br,pi0_px_1_br,...,rho_px_2_br,rho_py_2_br,rho_pz_2_br,aco_angle_1,y_1_1,y_1_2,w_a,w_b,m_1,m_2
entry,,,,,,,,,,,,,,,,,,,,,
8,38.866249,0.429429,-0.133119,38.863398,40.456445,-0.509745,-0.094749,-40.452882,8.569687,-0.429429,...,2.442491e-14,5.329071e-14,-47.420221,2.657029,-0.637964,-0.708372,1.228852,0.950417,1.220952,1.512969
25,17.175150,0.213861,0.469040,17.166846,9.529103,0.210109,-0.089910,-9.525340,29.088943,-0.213861,...,5.329071e-15,-1.154632e-14,-46.250908,1.547246,0.254573,0.594557,0.124674,1.936855,1.104476,0.660891
27,8.526993,0.209425,0.197722,8.520985,10.630995,-0.242771,-0.036193,-10.627246,37.577339,-0.209425,...,2.131628e-14,7.105427e-15,-46.096978,3.750274,0.638590,0.529623,1.461517,0.400455,0.823420,0.669092
45,41.836730,-0.050458,-0.144726,41.836217,2.599469,0.099976,0.004286,-2.593793,11.395560,0.050458,...,4.352074e-14,-3.197442e-14,-53.229947,1.192612,-0.572846,0.899114,0.059870,0.061072,0.499415,0.796268
50,23.455746,0.115471,-0.337611,23.452617,25.232731,-0.068646,0.327903,-25.230121,21.066138,-0.115471,...,-1.143530e-14,-9.547918e-15,-44.515301,3.264792,-0.054617,-0.132105,0.654131,0.931771,0.765599,0.730365


In [9]:
%%time
np.c_[df.pi_E_1_br, df.pi_px_1_br]

Wall time: 10 ms


array([[3.88662492e+01, 4.29429408e-01],
       [1.71751504e+01, 2.13860954e-01],
       [8.52699315e+00, 2.09425230e-01],
       ...,
       [4.80889579e+01, 1.09017298e-01],
       [1.97226794e+00, 2.75777872e-02],
       [1.50065016e+01, 3.16749176e-01]])

# test 2 - need to know exact 4 vector to do boosts properly

In [2]:
import numpy as np
from pylorentz import Momentum4

In [12]:
def getP(mom4):
    arr = np.array([mom4.p_x, mom4.p_y, mom4.p_z])
    return arr/np.linalg.norm(arr)

In [36]:
p_1 = Momentum4(1,100,100,100)
p_2 = Momentum4(1,1,1,-1)
angle = np.arccos(getP(p_1).dot(getP(p_2)))
print(f'angle: {angle}')

angle: 1.2309594173407747


In [37]:
rest_frame = Momentum4(10,1,-1,1)
boost = Momentum4(rest_frame[0], -rest_frame[1], -rest_frame[2], -rest_frame[3])
p_1_boosted = p_1.boost_particle(boost)
p_2_boosted = p_2.boost_particle(boost)

In [38]:
angle_boost = np.arccos(getP(p_1_boosted).dot(getP(p_2_boosted)))
print(f'angle: {angle_boost}')

angle: 1.28273889442413


In [14]:
getP(p_1)

array([0.57735027, 0.57735027, 0.57735027])

## test 3 - getting rotated gen neutrinos

In [7]:
import uproot
import pandas as pd
import numpy as np
from pylorentz import Momentum4
m_tau = 1.776

In [7]:
variables = [
        "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand",
        "aco_angle_1", "aco_angle_5", "aco_angle_6", "aco_angle_7",
        "mva_dm_1", "mva_dm_2",
        "tau_decay_mode_1", "tau_decay_mode_2",
        "ip_x_1", "ip_y_1", "ip_z_1", "ip_x_2", "ip_y_2", "ip_z_2",  # ignore impact parameter for now
        "pi_E_1", "pi_px_1", "pi_py_1", "pi_pz_1",
        "pi_E_2", "pi_px_2", "pi_py_2", "pi_pz_2",
        "pi0_E_1", "pi0_px_1", "pi0_py_1", "pi0_pz_1",
        "pi0_E_2", "pi0_px_2", "pi0_py_2", "pi0_pz_2",
        "y_1_1", "y_1_2",
        'met', 'metx', 'mety',
        'metcov00', 'metcov01', 'metcov10', 'metcov11',
        #             'sv_x_1', 'sv_y_1', 'sv_z_1', 'sv_x_2', 'sv_y_2','sv_z_2'
    ]
variables += ["gen_nu_p_1", "gen_nu_phi_1", "gen_nu_eta_1", #leading neutrino, gen level
            "gen_nu_p_2", "gen_nu_phi_2", "gen_nu_eta_2" #subleading neutrino, gen level
             ]
tree_tt = uproot.open("E:\Documents\Imperial\MSci_Project\MSci-Project-HiggsTauTau\shared\MVAFILE_AllHiggs_tt.root")["ntuple"]
df = tree_tt.pandas.df(variables)

In [9]:
df.columns

Index(['wt_cp_sm', 'wt_cp_ps', 'wt_cp_mm', 'rand', 'aco_angle_1',
       'aco_angle_5', 'aco_angle_6', 'aco_angle_7', 'mva_dm_1', 'mva_dm_2',
       'tau_decay_mode_1', 'tau_decay_mode_2', 'ip_x_1', 'ip_y_1', 'ip_z_1',
       'ip_x_2', 'ip_y_2', 'ip_z_2', 'pi_E_1', 'pi_px_1', 'pi_py_1', 'pi_pz_1',
       'pi_E_2', 'pi_px_2', 'pi_py_2', 'pi_pz_2', 'pi0_E_1', 'pi0_px_1',
       'pi0_py_1', 'pi0_pz_1', 'pi0_E_2', 'pi0_px_2', 'pi0_py_2', 'pi0_pz_2',
       'y_1_1', 'y_1_2', 'met', 'metx', 'mety', 'metcov00', 'metcov01',
       'metcov10', 'metcov11', 'gen_nu_p_1', 'gen_nu_phi_1', 'gen_nu_eta_1',
       'gen_nu_p_2', 'gen_nu_phi_2', 'gen_nu_eta_2'],
      dtype='object')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10103610 entries, 0 to 10103609
Data columns (total 49 columns):
 #   Column            Dtype  
---  ------            -----  
 0   wt_cp_sm          float64
 1   wt_cp_ps          float64
 2   wt_cp_mm          float64
 3   rand              float64
 4   aco_angle_1       float64
 5   aco_angle_5       float64
 6   aco_angle_6       float64
 7   aco_angle_7       float64
 8   mva_dm_1          int32  
 9   mva_dm_2          int32  
 10  tau_decay_mode_1  int32  
 11  tau_decay_mode_2  int32  
 12  ip_x_1            float64
 13  ip_y_1            float64
 14  ip_z_1            float64
 15  ip_x_2            float64
 16  ip_y_2            float64
 17  ip_z_2            float64
 18  pi_E_1            float64
 19  pi_px_1           float64
 20  pi_py_1           float64
 21  pi_pz_1           float64
 22  pi_E_2            float64
 23  pi_px_2           float64
 24  pi_py_2           float64
 25  pi_pz_2           float64
 26  pi0_E_1     

In [12]:
df.to_pickle('../shared/misc/df_tt_rho_rho_withgen.pkl')

In [15]:
nu_1 = Momentum4.m_eta_phi_p(np.zeros(len(df["gen_nu_phi_1"])), df["gen_nu_eta_1"], df["gen_nu_phi_1"], df["gen_nu_p_1"])

C:\Users\Stanley\.conda\envs\msci\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [27]:
nu_1[3][nu_1[3] == 9999] = np.nan

In [17]:
df = pd.read_pickle('../shared/df_tt_rho_rho.pkl')
df = df[(df['mva_dm_1'] == 1) & (df['mva_dm_2'] == 1) & (df["tau_decay_mode_1"] == 1) & (df["tau_decay_mode_2"] == 1)]

In [18]:
df.shape

(998268, 40)

In [13]:
df = pd.read_pickle('../shared/misc/df_tt_rho_rho_withgen.pkl')
df1 = df[(df['mva_dm_1'] == 1) & (df['mva_dm_2'] == 1) & (df["tau_decay_mode_1"] == 1) & (df["tau_decay_mode_2"] == 1)]

In [11]:
df2 = pd.read_pickle('../stanley/df_saved/rho_rho/df_rho_rho.pkl')

In [14]:
df1.shape

(998268, 49)

In [12]:
df2.shape

(998268, 27)

In [15]:
df3 = pd.read_pickle('../shared/input_df_reco/input_rho_rho.pkl')

In [25]:
df3.shape

(998268, 31)

In [24]:
df3.columns

Index(['pi_E_1_br', 'pi_px_1_br', 'pi_py_1_br', 'pi_pz_1_br', 'pi_E_2_br',
       'pi_px_2_br', 'pi_py_2_br', 'pi_pz_2_br', 'pi0_E_1_br', 'pi0_px_1_br',
       'pi0_py_1_br', 'pi0_pz_1_br', 'pi0_E_2_br', 'pi0_px_2_br',
       'pi0_py_2_br', 'pi0_pz_2_br', 'rho_E_1_br', 'rho_px_1_br',
       'rho_py_1_br', 'rho_pz_1_br', 'rho_E_2_br', 'rho_px_2_br',
       'rho_py_2_br', 'rho_pz_2_br', 'aco_angle_1', 'y_1_1', 'y_1_2', 'w_a',
       'w_b', 'm_1', 'm_2'],
      dtype='object')

In [16]:
df = pd.read_pickle('../shared/misc/df_br.pkl')

In [12]:
df.columns

Index(['entry', 'pi_E_1_br', 'pi_px_1_br', 'pi_py_1_br', 'pi_pz_1_br',
       'pi_E_2_br', 'pi_px_2_br', 'pi_py_2_br', 'pi_pz_2_br', 'pi0_E_1_br',
       'pi0_px_1_br', 'pi0_py_1_br', 'pi0_pz_1_br', 'pi0_E_2_br',
       'pi0_px_2_br', 'pi0_py_2_br', 'pi0_pz_2_br', 'rho_E_1_br',
       'rho_px_1_br', 'rho_py_1_br', 'rho_pz_1_br', 'rho_E_2_br',
       'rho_px_2_br', 'rho_py_2_br', 'rho_pz_2_br', 'aco_angle_1', 'y_1_1',
       'y_1_2', 'w_a', 'w_b', 'm_1', 'm_2', 'y', 'alpha_1', 'alpha_2',
       'E_nu_1', 'E_nu_2', 'p_t_nu_1', 'p_t_nu_2', 'p_z_nu_1', 'p_z_nu_2',
       'flag'],
      dtype='object')

In [31]:
df['flag'] = np.where( (df['p_t_nu_1']==-1) | (df['alpha_1']==-1) | (df['p_t_nu_2']==-1) | (df['alpha_2']==-1) | (df['E_nu_1']<0) | (df['E_nu_2']<0), 0, 1)

In [32]:
df.head()

,entry,pi_E_1_br,pi_px_1_br,pi_py_1_br,pi_pz_1_br,pi_E_2_br,pi_px_2_br,pi_py_2_br,pi_pz_2_br,pi0_E_1_br,...,y,alpha_1,alpha_2,E_nu_1,E_nu_2,p_t_nu_1,p_t_nu_2,p_z_nu_1,p_z_nu_2,flag
0,27,-9.520570,-0.281322,0.061643,9.515190,-9.520570,0.281322,-0.061643,-9.515190,-41.965766,...,1.0,3.503546,0.809264,-180.362635,-33.418774,0.594328,1.071559,180.361656,-33.401590,0
1,74,-15.212249,-0.285148,0.007680,15.208934,-15.212249,0.285148,-0.007680,-15.208934,-56.900663,...,1.0,3.717141,1.987254,-268.040429,-68.124183,1.063017,0.966009,268.038321,-68.117334,0
2,138,-17.769221,0.134191,-0.341849,17.764878,-17.769221,-0.134191,0.341849,-17.764878,-34.984946,...,1.0,0.338910,0.740946,-17.897843,-40.656159,0.869873,-1.000000,17.876692,-40.678914,0
3,177,-23.653185,-0.363155,-0.007247,23.649985,-23.653185,0.363155,0.007247,-23.649985,-51.433946,...,1.0,-2.188768,0.330466,164.311542,-10.552404,-1.000000,0.904721,-164.338085,-10.513549,0
4,184,-32.404365,0.419282,0.107584,32.401174,-32.404365,-0.419282,-0.107584,-32.401174,-12.539431,...,1.0,-1.916185,0.355058,86.053289,-14.615731,-1.000000,0.969338,-86.098811,-14.583551,0


In [34]:
df['flag'].sum()/len(df['flag'])*100

0.02682503899753311

In [20]:
p_z_nu_1 = df.alpha_1[0]*(df.pi_pz_1_br[0] + df.pi0_pz_1_br[0])
i = 0
E_nu_1 = (125.18**2 - (df.pi_E_1_br[i] + df.pi0_E_1_br[i])**2 + (df.pi_pz_1_br[i] + df.pi0_pz_1_br[i])** 2 + 2*p_z_nu_1*(df.pi_pz_1_br[i] + df.pi0_pz_1_br[i]))/(2*(df.pi_E_1_br[i] + df.pi0_E_1_br[i]))
p_z_nu_1
E_nu_1

194.85291640160656

In [21]:
p_t_nu_1 = np.sqrt(np.array(E_nu_1)**2 - np.array(p_z_nu_1)**2)

<ipython-input-21-c4efc2a59d34>:1: RuntimeWarning: invalid value encountered in sqrt
  p_t_nu_1 = np.sqrt(np.array(E_nu_1)**2 - np.array(p_z_nu_1)**2)


In [22]:
np.abs(E_nu_1) < np.abs(p_z_nu_1)

True

In [3]:
alpha_1 = np.load('../shared/alpha_analysis/alpha_1_100_b.npy')

In [7]:
(alpha_1==-1).sum()

45671

### from previous runs

In [80]:
alpha_1_old = np.load('./alpha_analysis/alpha_1_1000.npy')
alpha_2_old = np.load('./alpha_analysis/alpha_2_1000.npy')

In [81]:
alpha_1_old.shape

(998268,)

In [82]:
df = pd.read_pickle('..\shared\input_df_reco\input_rho_rho.pkl')

In [83]:
df.head()

,pi_E_1_br,pi_px_1_br,pi_py_1_br,pi_pz_1_br,pi_E_2_br,pi_px_2_br,pi_py_2_br,pi_pz_2_br,pi0_E_1_br,pi0_px_1_br,...,rho_px_2_br,rho_py_2_br,rho_pz_2_br,aco_angle_1,y_1_1,y_1_2,w_a,w_b,m_1,m_2
entry,,,,,,,,,,,,,,,,,,,,,
8,38.866249,0.429429,-0.133119,38.863398,40.456445,-0.509745,-0.094749,-40.452882,8.569687,-0.429429,...,2.442491e-14,5.329071e-14,-47.420221,2.657029,-0.637964,-0.708372,1.228852,0.950417,1.220952,1.512969
25,17.175150,0.213861,0.469040,17.166846,9.529103,0.210109,-0.089910,-9.525340,29.088943,-0.213861,...,5.329071e-15,-1.154632e-14,-46.250908,1.547246,0.254573,0.594557,0.124674,1.936855,1.104476,0.660891
27,8.526993,0.209425,0.197722,8.520985,10.630995,-0.242771,-0.036193,-10.627246,37.577339,-0.209425,...,2.131628e-14,7.105427e-15,-46.096978,3.750274,0.638590,0.529623,1.461517,0.400455,0.823420,0.669092
45,41.836730,-0.050458,-0.144726,41.836217,2.599469,0.099976,0.004286,-2.593793,11.395560,0.050458,...,4.352074e-14,-3.197442e-14,-53.229947,1.192612,-0.572846,0.899114,0.059870,0.061072,0.499415,0.796268
50,23.455746,0.115471,-0.337611,23.452617,25.232731,-0.068646,0.327903,-25.230121,21.066138,-0.115471,...,-1.143530e-14,-9.547918e-15,-44.515301,3.264792,-0.054617,-0.132105,0.654131,0.931771,0.765599,0.730365


In [84]:

df['alpha_1'] = alpha_1_old
df['alpha_2'] = alpha_2_old
df['p_z_nu_1'] = alpha_1_old*(df.pi_pz_1_br + df.pi0_pz_1_br)
df['p_z_nu_2'] = alpha_2_old*(df.pi_pz_2_br + df.pi0_pz_2_br)
df['E_nu_1'] = (m_tau**2 - (df.pi_E_1_br+df.pi0_E_1_br)**2 + (df.pi_pz_1_br + df.pi0_pz_1_br)** 2 + 2*df.p_z_nu_1*(df.pi_pz_1_br + df.pi0_pz_1_br))/(2*(df.pi_E_1_br+df.pi0_E_1_br))
df['E_nu_2'] = (m_tau**2 - (df.pi_E_2_br+df.pi0_E_2_br)**2 + (df.pi_pz_2_br + df.pi0_pz_2_br)** 2 + 2*df.p_z_nu_2*(df.pi_pz_2_br + df.pi0_pz_2_br))/(2*(df.pi_E_2_br+df.pi0_E_2_br))
df['p_t_nu_1'] = np.sqrt(np.array(df.E_nu_1)**2 - np.array(df.p_z_nu_1)**2)
df['p_t_nu_2'] = np.sqrt(np.array(df.E_nu_2)**2 - np.array(df.p_z_nu_2)**2)
# df_red = df[(df['alpha_1'] > 0) & (df['alpha_2'] > 0) & (
#             df['E_nu_1'] > 0) & (df['E_nu_2'] > 0)].reset_index(drop=True)

df_red = df[(df['alpha_1'] > 0) & (df['alpha_2'] > 0) & (
            df['E_nu_1'] > 0) & (df['E_nu_2'] > 0) & (~np.isnan(df['p_t_nu_1'])) & (~np.isnan(df['p_t_nu_2']))].reset_index(drop=True)

<ipython-input-84-299a0640d27c>:8: RuntimeWarning: invalid value encountered in sqrt
  df['p_t_nu_1'] = np.sqrt(np.array(df.E_nu_1)**2 - np.array(df.p_z_nu_1)**2)
<ipython-input-84-299a0640d27c>:9: RuntimeWarning: invalid value encountered in sqrt
  df['p_t_nu_2'] = np.sqrt(np.array(df.E_nu_2)**2 - np.array(df.p_z_nu_2)**2)


In [85]:
df.head()

,pi_E_1_br,pi_px_1_br,pi_py_1_br,pi_pz_1_br,pi_E_2_br,pi_px_2_br,pi_py_2_br,pi_pz_2_br,pi0_E_1_br,pi0_px_1_br,...,m_1,m_2,alpha_1,alpha_2,p_z_nu_1,p_z_nu_2,E_nu_1,E_nu_2,p_t_nu_1,p_t_nu_2
entry,,,,,,,,,,,,,,,,,,,,,
8,38.866249,0.429429,-0.133119,38.863398,40.456445,-0.509745,-0.094749,-40.452882,8.569687,-0.429429,...,1.220952,1.512969,-1.000000,-1.000000,-47.420221,47.420221,-47.386977,-47.386986,NaN,NaN
25,17.175150,0.213861,0.469040,17.166846,9.529103,0.210109,-0.089910,-9.525340,29.088943,-0.213861,...,1.104476,0.660891,1.405986,5.760983,65.028118,-266.450680,65.030490,266.452856,0.555376,1.076724
27,8.526993,0.209425,0.197722,8.520985,10.630995,-0.242771,-0.036193,-10.627246,37.577339,-0.209425,...,0.823420,0.669092,7.170637,2.127554,330.544686,-98.073792,330.518818,98.092816,NaN,1.931799
45,41.836730,-0.050458,-0.144726,41.836217,2.599469,0.099976,0.004286,-2.593793,11.395560,0.050458,...,0.499415,0.796268,0.825470,9.321733,43.939720,-496.195347,43.965070,496.163508,1.492779,NaN
50,23.455746,0.115471,-0.337611,23.452617,25.232731,-0.068646,0.327903,-25.230121,21.066138,-0.115471,...,0.765599,0.730365,2.691481,3.238841,119.812080,-144.177972,119.823204,144.188002,1.632730,1.700731


In [86]:
print(df.shape, df_red.shape)

(998268, 39) (629235, 39)


### from old code on new file

In [35]:
df_old = pd.read_pickle('../shared/misc/df_br_old.pkl')

In [78]:
df_red = df_old[(df_old['alpha_1'] > 0) & (df_old['alpha_2'] > 0) & (
            df_old['E_nu_1'] > 0) & (df_old['E_nu_2'] > 0) & (~np.isnan(df_old['p_t_nu_1'])) & (~np.isnan(df_old['p_t_nu_2']))].reset_index(drop=True)

## testing current code with old code

In [12]:
alpha_1 = np.load('../shared/alpha_analysis/alpha_1_1000.npy')
alpha_2 = np.load('../shared/alpha_analysis/alpha_2_1000.npy')

In [13]:
df = pd.read_pickle('../shared/input_df_reco/input_rho_rho.pkl')

In [14]:
p_z_nu_1 = alpha_1*(df.pi_pz_1_br + df.pi0_pz_1_br)
p_z_nu_2 = alpha_2*(df.pi_pz_2_br + df.pi0_pz_2_br)
E_nu_1 = (m_tau**2 - (df.pi_E_1_br+df.pi0_E_1_br)**2 + (df.pi_pz_1_br + df.pi0_pz_1_br)
          ** 2 + 2*p_z_nu_1*(df.pi_pz_1_br + df.pi0_pz_1_br))/(2*(df.pi_E_1_br+df.pi0_E_1_br))
E_nu_2 = (m_tau**2 - (df.pi_E_2_br+df.pi0_E_2_br)**2 + (df.pi_pz_2_br + df.pi0_pz_2_br)
          ** 2 + 2*p_z_nu_2*(df.pi_pz_2_br + df.pi0_pz_2_br))/(2*(df.pi_E_2_br+df.pi0_E_2_br))
p_t_nu_1 = np.sqrt(np.array(E_nu_1)**2 - np.array(p_z_nu_1)**2)
p_t_nu_2 = np.sqrt(np.array(E_nu_2)**2 - np.array(p_z_nu_2)**2)
df['alpha_1'] = alpha_1
df['alpha_2'] = alpha_2
df['E_nu_1'] = E_nu_1
df['E_nu_2'] = E_nu_2
df['p_t_nu_1'] = p_t_nu_1
df['p_t_nu_2'] = p_t_nu_2
df['p_z_nu_1'] = p_z_nu_1
df['p_z_nu_2'] = p_z_nu_2
df_red = df[(df['alpha_1'] > 0) & (df['alpha_2'] > 0) & (
            df['E_nu_1'] > 0) & (df['E_nu_2'] > 0) & (~np.isnan(df['p_t_nu_1'])) & (~np.isnan(df['p_t_nu_2']))].reset_index(drop=True)

<ipython-input-14-5c75a7885f98>:7: RuntimeWarning: invalid value encountered in sqrt
  p_t_nu_1 = np.sqrt(np.array(E_nu_1)**2 - np.array(p_z_nu_1)**2)
<ipython-input-14-5c75a7885f98>:8: RuntimeWarning: invalid value encountered in sqrt
  p_t_nu_2 = np.sqrt(np.array(E_nu_2)**2 - np.array(p_z_nu_2)**2)


In [15]:
df.head()

,pi_E_1_br,pi_px_1_br,pi_py_1_br,pi_pz_1_br,pi_E_2_br,pi_px_2_br,pi_py_2_br,pi_pz_2_br,pi0_E_1_br,pi0_px_1_br,...,m_1,m_2,alpha_1,alpha_2,E_nu_1,E_nu_2,p_t_nu_1,p_t_nu_2,p_z_nu_1,p_z_nu_2
entry,,,,,,,,,,,,,,,,,,,,,
8,38.866249,0.429429,-0.133119,38.863398,40.456445,-0.509745,-0.094749,-40.452882,8.569687,-0.429429,...,1.220952,1.512969,-1.000000,-1.000000,-47.386977,-47.386986,NaN,NaN,-47.420221,47.420221
25,17.175150,0.213861,0.469040,17.166846,9.529103,0.210109,-0.089910,-9.525340,29.088943,-0.213861,...,1.104476,0.660891,1.405986,5.760983,65.030490,266.452856,0.555376,1.076724,65.028118,-266.450680
27,8.526993,0.209425,0.197722,8.520985,10.630995,-0.242771,-0.036193,-10.627246,37.577339,-0.209425,...,0.823420,0.669092,2.929308,0.525376,135.037562,24.245046,1.198206,1.139712,135.032246,-24.218243
45,41.836730,-0.050458,-0.144726,41.836217,2.599469,0.099976,0.004286,-2.593793,11.395560,0.050458,...,0.499415,0.796268,0.497134,1.975634,26.488511,105.174797,1.176025,1.582437,26.462392,-105.162891
50,23.455746,0.115471,-0.337611,23.452617,25.232731,-0.068646,0.327903,-25.230121,21.066138,-0.115471,...,0.765599,0.730365,2.691481,3.238841,119.823204,144.188002,1.632730,1.700731,119.812080,-144.177972


In [16]:
df_red.head()

,pi_E_1_br,pi_px_1_br,pi_py_1_br,pi_pz_1_br,pi_E_2_br,pi_px_2_br,pi_py_2_br,pi_pz_2_br,pi0_E_1_br,pi0_px_1_br,...,m_1,m_2,alpha_1,alpha_2,E_nu_1,E_nu_2,p_t_nu_1,p_t_nu_2,p_z_nu_1,p_z_nu_2
0,17.175150,0.213861,0.469040,17.166846,9.529103,0.210109,-0.089910,-9.525340,29.088943,-0.213861,...,1.104476,0.660891,1.405986,5.760983,65.030490,266.452856,0.555376,1.076724,65.028118,-266.450680
1,8.526993,0.209425,0.197722,8.520985,10.630995,-0.242771,-0.036193,-10.627246,37.577339,-0.209425,...,0.823420,0.669092,2.929308,0.525376,135.037562,24.245046,1.198206,1.139712,135.032246,-24.218243
2,41.836730,-0.050458,-0.144726,41.836217,2.599469,0.099976,0.004286,-2.593793,11.395560,0.050458,...,0.499415,0.796268,0.497134,1.975634,26.488511,105.174797,1.176025,1.582437,26.462392,-105.162891
3,23.455746,0.115471,-0.337611,23.452617,25.232731,-0.068646,0.327903,-25.230121,21.066138,-0.115471,...,0.765599,0.730365,2.691481,3.238841,119.823204,144.188002,1.632730,1.700731,119.812080,-144.177972
4,5.760700,0.068916,0.228395,5.754067,21.168171,-0.077786,0.321893,-21.165121,36.728783,-0.068916,...,0.806553,0.716946,1.007152,0.049665,42.807425,2.140627,1.364509,0.361641,42.785672,-2.109857


In [17]:
print(df.shape, df_red.shape)

(998268, 39) (862747, 39)


## test 4 - playing around with eqn 26

In [2]:
import numpy as np